# 构建神经网络的三种方法
## 1. `nn.Sequential`

In [1]:
import torch
import torch.nn as nn
import numpy as np
import os

In [2]:
seq_model = nn.Sequential(nn.Linear(1, 11),
                         nn.Tanh(),
                         nn.Linear(11, 1))
seq_model

Sequential(
  (0): Linear(in_features=1, out_features=11, bias=True)
  (1): Tanh()
  (2): Linear(in_features=11, out_features=1, bias=True)
)

## 2. 添加名字更清晰
使用`OrderedDict`给每一个层添加一个label

In [3]:
from collections import OrderedDict

In [4]:
namedseq_model = nn.Sequential(OrderedDict([
    ('hidden_linear', nn.Linear(1, 12)),
    ('hidden_activation', nn.Tanh()),
    ('output_linear', nn.Linear(12, 1))
]))
namedseq_model

Sequential(
  (hidden_linear): Linear(in_features=1, out_features=12, bias=True)
  (hidden_activation): Tanh()
  (output_linear): Linear(in_features=12, out_features=1, bias=True)
)

使用 nn.Sequential，你确实没有办法做到更多的控制，一切都是按照顺序来的

## 3. 使用nn.Module控制更多
如果使用 nn.Module，那么你就可以对这些数据处理的过程做到完全的控制

In [5]:
class SubclassModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden_linear = nn.Linear(1, 13)
        self.hidden_activation = nn.Tanh()
        self.output_linear = nn.Linear(13, 1)
        
    def forward(self, input):
        hidden_t = self.hidden_linear(input)
        acticated_t = self.hidden_activation(hidden_t)
        output_t = self.output_linear(activated_t)
        
        return output_t
    
subclass_model = SubclassModel()
subclass_model

SubclassModel(
  (hidden_linear): Linear(in_features=1, out_features=13, bias=True)
  (hidden_activation): Tanh()
  (output_linear): Linear(in_features=13, out_features=1, bias=True)
)

相对来说，这种方式看上去要比前面的两种方式要繁琐一点  
但是这种方式也能提供更多的灵活性

In [7]:
for type_str, model in [('seq', seq_model), ('namedseq', namedseq_model), ('subclass', subclass_model)]:
    print(type_str)
    for name_str, param in model.named_parameters():
        print("{:21}{:19}{}".format(name_str, str(param.shape), param.numel()))
        
    print()

seq
0.weight             torch.Size([11, 1])11
0.bias               torch.Size([11])   11
2.weight             torch.Size([1, 11])11
2.bias               torch.Size([1])    1

namedseq
hidden_linear.weight torch.Size([12, 1])12
hidden_linear.bias   torch.Size([12])   12
output_linear.weight torch.Size([1, 12])12
output_linear.bias   torch.Size([1])    1

subclass
hidden_linear.weight torch.Size([13, 1])13
hidden_linear.bias   torch.Size([13])   13
output_linear.weight torch.Size([1, 13])13
output_linear.bias   torch.Size([1])    1



instance 例子、实例  
utility 效用、裨益  
instantiate 实例化  
appetite 食欲、胃口

在上面的代码中，我们要在 subcalssmodel中定义子模块，或者说对一些子函数进行声明  
- 有的函数是有参数的，这种声明倒还是比较有必要的  
- 有的函数根本就没有参数，这种声明就没什么必要

In [9]:
class SubclassFunctionModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.hidden_linear = nn.Linear(1, 14)
        
        self.output_linear = nn.Linear(14, 1)        
        
    def forward(self, input):
        hidden_t = self.hidden_linear(input)
        activated_t = torch.tanh(hidden_t)
        output_t = self.output_linear(activated_t)
        
        return output_t
    
func_model = SubclassFunctionModel()
func_model

SubclassFunctionModel(
  (hidden_linear): Linear(in_features=1, out_features=14, bias=True)
  (output_linear): Linear(in_features=14, out_features=1, bias=True)
)

注意到在这个类的代码中，我们不再重新定义 tanh函数，而是直接使用`torch.tanh`函数